# Feathr Fraud Detection Sample

This notebook illustrates the use of Feature Store to create a model that predicts the fraud status of transactions based on the user account data and trasaction data. All the data that was used in the notebook can be found here: https://github.com/microsoft/r-server-fraud-detection.


In the following Notebook, we 
1. Install the latest Feathr code (to include some unreleased features) 
2. Define Environment Variables & `yaml_config` Settings 
3. Create `FeathrClient` and Define `FeatureAnchor`
4. `build_features` and `get_offline_features`
5. Visualize features and train Fraud Detection Model
6. `materialize_features` and `get_online_features`

## Setup Feathr Developer Environment

>Prior to running the notebook, if you have not deployed all the required resources, please refer to the guide here and follow the steps to do so: https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html

In [ ]:
# Install feathr from the latest codes in the repo. You may use `pip install feathr[notebook]` as well.
# !pip install "git+https://github.com/feathr-ai/feathr.git#subdirectory=feathr_project&egg=feathr[notebook]"  

In [ ]:
from datetime import datetime, timedelta
import os
from pathlib import Path

import pandas as pd

import feathr
from feathr import (
    FeathrClient,
    STRING, BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    FRAUD_DETECTION_ACCOUNT_INFO_URL,
    FRAUD_DETECTION_FRAUD_TRANSACTIONS_URL,
    FRAUD_DETECTION_UNTAGGED_TRANSACTIONS_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df
from feathr.utils.platform import is_databricks


print(f"Feathr version: {feathr.__version__}")

In [ ]:
RESOURCE_PREFIX = ""  # TODO fill the value used to deploy the resources via ARM template
PROJECT_NAME = "fraud_detection"

# Currently support: 'azure_synapse', 'databricks', and 'local' 
SPARK_CLUSTER = "local"

# TODO fill values to use databricks cluster:
DATABRICKS_CLUSTER_ID = None             # Set Databricks cluster id to use an existing cluster
if is_databricks():
    # If this notebook is running on Databricks, its context can be used to retrieve token and instance URL
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    DATABRICKS_WORKSPACE_TOKEN_VALUE = ctx.apiToken().get()
    SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL = f"https://{ctx.tags().get('browserHostName').get()}"
else:
    DATABRICKS_WORKSPACE_TOKEN_VALUE = None                  # Set Databricks workspace token to use databricks
    SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL = None  # Set Databricks workspace url to use databricks

# TODO fill values to use Azure Synapse cluster:
AZURE_SYNAPSE_SPARK_POOL = None  # Set Azure Synapse Spark pool name
AZURE_SYNAPSE_URL = None         # Set Azure Synapse workspace url to use Azure Synapse
ADLS_KEY = None                  # Set Azure Data Lake Storage key to use Azure Synapse

USE_CLI_AUTH = False  # Set to True to use CLI authentication

# An existing Feathr config file path. If None, we'll generate a new config based on the constants in this cell.
FEATHR_CONFIG_PATH = None

# (For the notebook test pipeline) If true, use ScrapBook package to collect the results.
SCRAP_RESULTS = False

In [ ]:
if SPARK_CLUSTER == "azure_synapse" and not os.environ.get("ADLS_KEY"):
    os.environ["ADLS_KEY"] = ADLS_KEY
elif SPARK_CLUSTER == "databricks" and not os.environ.get("DATABRICKS_WORKSPACE_TOKEN_VALUE"):
    os.environ["DATABRICKS_WORKSPACE_TOKEN_VALUE"] = DATABRICKS_WORKSPACE_TOKEN_VALUE

## Permission
To run the cells below, you need additional permission: permission to your managed identity to access the keyvault, and permission to the user to access the Storage Blob. Run the following lines of command in the Cloud Shell in order to grant yourself the access.

```
userId=<email_id_of_account_requesting_access>
resource_prefix=<resource_prefix>
synapse_workspace_name="${resource_prefix}syws"
keyvault_name="${resource_prefix}kv"
objectId=$(az ad user show --id $userId --query id -o tsv)
az keyvault update --name $keyvault_name --enable-rbac-authorization false
az keyvault set-policy -n $keyvault_name --secret-permissions get list --object-id $objectId
az role assignment create --assignee $userId --role "Storage Blob Data Contributor"
az synapse role assignment create --workspace-name $synapse_workspace_name --role "Synapse Contributor" --assignee $userId
```

In [ ]:
# Get an authentication credential to access Azure resources and register features
if USE_CLI_AUTH:
    # Use AZ CLI interactive browser authentication
    !az login --use-device-code
    from azure.identity import AzureCliCredential
    credential = AzureCliCredential(additionally_allowed_tenants=['*'],)
elif "AZURE_TENANT_ID" in os.environ and "AZURE_CLIENT_ID" in os.environ and "AZURE_CLIENT_SECRET" in os.environ:
    # Use Environment variable secret
    from azure.identity import EnvironmentCredential
    credential = EnvironmentCredential()
else:
    # Try to use the default credential
    from azure.identity import DefaultAzureCredential
    credential = DefaultAzureCredential(
        exclude_interactive_browser_credential=False,
        additionally_allowed_tenants=['*'],
    )

In [ ]:
# Redis password
if 'REDIS_PASSWORD' not in os.environ:
    from azure.keyvault.secrets import SecretClient
    vault_url = f"https://{RESOURCE_PREFIX}kv.vault.azure.net"
    secret_client = SecretClient(vault_url=vault_url, credential=credential)
    retrieved_secret = secret_client.get_secret('FEATHR-ONLINE-STORE-CONN').value
    os.environ['REDIS_PASSWORD'] = retrieved_secret.split(",")[1].split("password=", 1)[1]

In [ ]:
if FEATHR_CONFIG_PATH:
    config_path = FEATHR_CONFIG_PATH
else:
    config_path = generate_config(
        resource_prefix=RESOURCE_PREFIX,
        project_name=PROJECT_NAME,
        spark_config__spark_cluster=SPARK_CLUSTER,
        spark_config__azure_synapse__dev_url=AZURE_SYNAPSE_URL,
        spark_config__azure_synapse__pool_name=AZURE_SYNAPSE_SPARK_POOL,
        spark_config__databricks__workspace_instance_url=SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL,
        databricks_cluster_id=DATABRICKS_CLUSTER_ID,
    )

with open(config_path, 'r') as f: 
    print(f.read())

## Initialize Feathr Client

In [ ]:
client = FeathrClient(config_path=config_path, credential=credential)

## Prepare Datasets

1. Download Account info data, fraud transactions data, and untagged transactions data.
2. Merge two transactions data (fraud and untagged) into one
3. Upload data files to cloud so that the target cluster can consume

In [ ]:
# Use dbfs if the notebook is running on Databricks
if is_databricks():
    WORKING_DIR = f"/dbfs/{PROJECT_NAME}"
else:
    WORKING_DIR = PROJECT_NAME

In [ ]:
# Download datasets
account_info_file_path = f"{WORKING_DIR}/account_info.csv"
fraud_transactions_file_path = f"{WORKING_DIR}/fraud_transactions.csv"
obs_transactions_file_path = f"{WORKING_DIR}/obs_transactions.csv"
maybe_download(
    src_url=FRAUD_DETECTION_ACCOUNT_INFO_URL,
    dst_filepath=account_info_file_path,
)
maybe_download(
    src_url=FRAUD_DETECTION_FRAUD_TRANSACTIONS_URL,
    dst_filepath=fraud_transactions_file_path,
)
maybe_download(
    src_url=FRAUD_DETECTION_UNTAGGED_TRANSACTIONS_URL,
    dst_filepath=obs_transactions_file_path,
)

In [ ]:
# Concat fraud and obs transactions
fraud_df = pd.read_csv(fraud_transactions_file_path)
fraud_df["fraud_tag"] = "Fraud"
obs_df = pd.read_csv(obs_transactions_file_path)
obs_df["fraud_tag"] = "Unknown"

transactions_file_path = f"{WORKING_DIR}/transactions.csv"
transactions_df = pd.concat([fraud_df, obs_df], ignore_index=True)
transactions_df.to_csv(transactions_file_path, index=False)

In [ ]:
# Upload files to cloud if needed
if client.spark_runtime == "local":
    # In local mode, we can use the same data path as the source.
    # If the notebook is running on databricks, DATA_FILE_PATH should be already a dbfs path.
    account_info_source_path = account_info_file_path
    transactions_source_path = transactions_file_path
elif client.spark_runtime == "databricks" and is_databricks():
    # If the notebook is running on databricks, we can use the same data path as the source.
    account_info_source_path = account_info_file_path.replace("/dbfs", "dbfs:")
    transactions_source_path = transactions_file_path.replace("/dbfs", "dbfs:")
else:
    # Otherwise, upload the local file to the cloud storage (either dbfs or adls).
    account_info_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(account_info_file_path)
    transactions_source_path = client.feathr_spark_launcher.upload_or_get_cloud_path(transactions_file_path)

## Define Features

Now, we define following features:
- Account features: Account-level features that will be joined to observation data on accountID
- Transaction features: The features that will be joined to observation data on transactionID
- Transaction aggregated features: The features aggregated by accountID
- Derived features: The features derived from other features

Some important concepts include `HdfsSource`, `TypedKey`, `Feature`, `FeatureAnchor`, and `DerivedFeature`. Please refer to feathr [documents](https://feathr.readthedocs.io/en/latest/feathr.html) to learn more about the details.


### Account Features

In [ ]:
# Check account data
pd.read_csv(account_info_file_path).head()

In [ ]:
def account_dropna(df):
    """Drop rows with missing values in the account info dataset."""
    return df.select(
        "accountID",
        "transactionDate",
        "accountCountry",
        "isUserRegistered",
        "numPaymentRejects1dPerUser",
        "accountAge",
    ).dropna()


account_info_source = HdfsSource(
    name="account_data",
    path=account_info_source_path,
    event_timestamp_column="transactionDate",
    timestamp_format="yyyyMMdd",
    preprocessing=account_dropna,
)

In [ ]:
# Account features will be joined to observation data on accountID
account_id = TypedKey(
    key_column="accountID",
    key_column_type=ValueType.STRING,
    description="account id",
)
                
account_features = [
    Feature(
        name="account_country",
        key=account_id,
        feature_type=STRING, 
        transform="accountCountry",
    ),
    Feature(
        name="is_user_registered",
        key=account_id,
        feature_type=BOOLEAN,
        transform="isUserRegistered==TRUE",
    ),
    Feature(
        name="num_payment_rejects_1d_per_user",
        key=account_id,
        feature_type=INT32,
        transform="numPaymentRejects1dPerUser",
    ),
    Feature(
        name="account_age",
        key=account_id,
        feature_type=INT32,
        transform="accountAge",
    ),
]

account_anchor = FeatureAnchor(
    name="account_features",
    source=account_info_source,
    features=account_features,
)

### Transaction Features

In [ ]:
# Check transaction data
pd.read_csv(transactions_file_path).head()

In [ ]:
def transaction_dropna(df):
    """Drop rows with missing values in the transactions dataset."""
    return df.dropna(subset=[
        "accountID",
        "transactionDate",
        "transactionID",
        "transactionCurrencyCode",
        "transactionAmount",
        "transactionTime",
        "ipCountryCode",
    ])


transactions_source = HdfsSource(
    name="transaction_data",
    path=transactions_source_path,
    event_timestamp_column="transactionDate",
    timestamp_format="yyyyMMdd",
    preprocessing=transaction_dropna,
)

In [ ]:
# Transaction features will be joined to observation data on transactionID
transaction_id = TypedKey(
    key_column="transactionID",
    key_column_type=ValueType.STRING,
    description="transaction id",
)

transaction_amount = Feature(
    name="transaction_amount",
    key=transaction_id,
    feature_type=FLOAT,
    transform="transactionAmount",
)

transaction_features = [
    transaction_amount,
    Feature(
        name="transaction_ip_country_code",
        key=transaction_id,
        feature_type=STRING,
        transform="ipCountryCode",
    ),
    Feature(
        name="transaction_currency_code",
        key=transaction_id,
        feature_type=STRING,
        transform="transactionCurrencyCode",
    ),
    Feature(
        name="transaction_time",
        key=transaction_id,
        feature_type=INT32,
        transform="transactionTime",
    ),
]

transaction_feature_anchor = FeatureAnchor(
    name="transaction_features",
    source=transactions_source,
    features=transaction_features,
)

### Transaction Aggregated Features

In [ ]:
# average amount of transaction in that week
avg_transaction_amount = Feature(
    name="avg_transaction_amount",
    key=account_id,
    feature_type=FLOAT,
    transform=WindowAggTransformation(
        agg_expr="cast_float(transactionAmount)", agg_func="AVG", window="7d"
    ),
)

agg_features = [
    avg_transaction_amount,
    # number of transaction that took place in a day
    Feature(
        name="num_transaction_count_in_day",
        key=account_id,
        feature_type=INT32,
        transform=WindowAggTransformation(
            agg_expr="transactionID", agg_func="COUNT", window="1d"
        ),
    ),
    # number of transaction that took place in the past week
    Feature(
        name="num_transaction_count_in_week",
        key=account_id,
        feature_type=INT32,
        transform=WindowAggTransformation(
            agg_expr="transactionID", agg_func="COUNT", window="7d"
        ),
    ),
    # amount of transaction that took place in a day
    Feature(
        name="total_transaction_amount_in_day",
        key=account_id,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="cast_float(transactionAmount)", agg_func="SUM", window="1d"
        ),
    ),
    # average time of transaction in the past week
    Feature(
        name="avg_transaction_time",
        key=account_id,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="cast_float(transactionTime)", agg_func="AVG", window="7d"
        ),
    ),
]

agg_anchor = FeatureAnchor(
    name="transaction_agg_features",
    source=transactions_source,
    features=agg_features,
)

### Derived Features

In [ ]:
derived_features = [
    DerivedFeature(
        name="feature_diff_current_and_avg_amount",
        key=[transaction_id, account_id],
        feature_type=FLOAT,
        input_features=[transaction_amount, avg_transaction_amount],
        transform="transaction_amount - avg_transaction_amount",
    )
]

## Build and Get Features

In [ ]:
client.build_features(
    anchor_list=[
        account_anchor,
        transaction_feature_anchor,
        agg_anchor,
    ],
    derived_feature_list=derived_features,
)

In [ ]:
account_feature_names = [feat.name for feat in account_features] + [feat.name for feat in agg_features]
transactions_feature_names = [feat.name for feat in transaction_features]
derived_feature_names = [feat.name for feat in derived_features]

In [ ]:
account_feature_query = FeatureQuery(
    feature_list=account_feature_names,
    key=account_id,
)

transactions_feature_query = FeatureQuery(
    feature_list=transactions_feature_names,
    key=transaction_id,
)

derived_feature_query = FeatureQuery(
    feature_list=derived_feature_names,
    key=[transaction_id, account_id],
)
                   
settings = ObservationSettings(
    observation_path=transactions_source_path,
    event_timestamp_column="transactionDate",
    timestamp_format="yyyyMMdd",
)
    
client.get_offline_features(
    observation_settings=settings,
    feature_query=[account_feature_query, transactions_feature_query, derived_feature_query],
    output_path=transactions_source_path.rpartition("/")[0] + f"/fraud_transactions_features.avro",
)

client.wait_job_to_finish(timeout_sec=5000)

In [ ]:
df = get_result_df(client)[
    account_feature_names
    + transactions_feature_names
    + derived_feature_names
    + ["accountID", "transactionID", "fraud_tag"]
]

# Data cleaning: Remove the records if the account does not exist in the account info dataset
df.dropna(subset=["is_user_registered"], inplace=True)

df.head(5)

In [ ]:
df.nunique()

### Feature Visualization

In [ ]:
import plotly.express as px


NUM_SAMPLES_TO_PLOT = 10000

In [ ]:
fig = px.scatter_matrix(
    df[:NUM_SAMPLES_TO_PLOT],
    dimensions=account_feature_names + transactions_feature_names + derived_feature_names,
    color="fraud_tag",
    title="Scatter matrix of transaction dataset",
)
fig.update_traces(diagonal_visible=False, marker_size=3)
fig.update_layout(
    width=1000,
    height=1000,
    font_size=5,
)
fig.show()

## Build Fraud Detection Model

In this notebook, we train one-class Support Vector Machine (SVM) to score the transactions, where the score results can be used to determine if the transactions are fraud or not.

### Feature Preprocessing

Before we input the features to the model, we convert categorical features into neumeric vectors (using a simple one-hot encoding technique) and do standard scaling all the features.

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
# Split fraud (train) and observation (test) datasets
fraud_df = df[df["fraud_tag"]=="Fraud"].drop(["accountID", "transactionID", "fraud_tag"], axis="columns") 
obs_df = df[df["fraud_tag"]=="Unknown"].drop(["accountID", "transactionID", "fraud_tag"], axis="columns") 
print(f"Num fraud samples = {len(fraud_df)}", f"Num untagged samples = {len(obs_df)}", sep="\n")

In [ ]:
# Feature names to encode
enc_feature_names = ["account_country", "is_user_registered", "transaction_ip_country_code", "transaction_currency_code"]

In [ ]:
# Define and fit one-hot encoder
enc = OneHotEncoder(handle_unknown="ignore").fit(fraud_df[enc_feature_names])

fraud_features = np.concatenate(
    (
        # Encoded features
        enc.transform(fraud_df[enc_feature_names]).toarray(),
        # Other features that don't need to be encoded
        fraud_df.drop(enc_feature_names, axis="columns").fillna(0).to_numpy(),
        
    ),
    axis=1,
)

# Define and fit standard scaler
scaler = StandardScaler().fit(fraud_features)

fraud_features = scaler.transform(fraud_features)
print(f"A sample of fraud feature:\n{fraud_features[0]}\nData shape = {fraud_features.shape}")

In [ ]:
obs_features = np.concatenate(
    (
        # Encoded features
        enc.transform(obs_df[enc_feature_names]).toarray(),
        # Other features that don't need to be encoded
        obs_df.drop(enc_feature_names, axis="columns").fillna(0).to_numpy(),
        
    ),
    axis=1,
)
obs_features = scaler.transform(obs_features)
print(f"A sample of observation feature:\n{obs_features[0]}\nData shape = {obs_features.shape}")

### Model Training

In [ ]:
from sklearn.svm import OneClassSVM

clf = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1).fit(fraud_features)

### Data Scoring

In [ ]:
fraud_feature_scores = clf.score_samples(fraud_features)
fraud_feature_scores

In [ ]:
obs_feature_scores = clf.score_samples(obs_features)
obs_feature_scores

### Result Visualization

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# We subsample observation transaction data for visualization
obs_sample_idx = np.random.choice(len(obs_features), size=NUM_SAMPLES_TO_PLOT, replace=False)
obs_sample_idx

In [ ]:
scores = np.concatenate([fraud_feature_scores, obs_feature_scores[obs_sample_idx]], axis=0)
scores.shape

In [ ]:
features = np.concatenate([fraud_features, obs_features[obs_sample_idx]], axis=0)
features.shape

In [ ]:
tsne = TSNE(n_components=2, perplexity=20, n_iter=300)
tsne_results = tsne.fit_transform(features)

In [ ]:
fig = px.scatter(x=tsne_results[:, 0], y=tsne_results[:, 1], color=scores)
fig.update_traces(marker_size=5, marker_opacity=0.5)
fig.update_layout(
    width=800,
    height=800,
)

## Materialize Features in Redis

Now, we materialize features to `RedisSink` so that we can retrieve online features.

In [ ]:
ACCOUNT_FEATURE_TABLE_NAME = "fraudDetectionAccountFeatures" 

backfill_time = BackfillTime(
    start=datetime(2013, 8, 4),
    end=datetime(2013, 8, 4),
    step=timedelta(days=1),
)

In [ ]:
client.materialize_features(
    MaterializationSettings(
        ACCOUNT_FEATURE_TABLE_NAME,
        backfill_time=backfill_time,
        sinks=[RedisSink(table_name=ACCOUNT_FEATURE_TABLE_NAME)],
        feature_names=account_feature_names[1:],
    ),
    allow_materialize_non_agg_feature=True,
)

client.wait_job_to_finish(timeout_sec=5000)

In [ ]:
materialized_feature_values = client.get_online_features(
    ACCOUNT_FEATURE_TABLE_NAME,
    key="A1055520452832600",
    feature_names=account_feature_names[1:],
)
materialized_feature_values

Scrap results for unit test

In [ ]:
if SCRAP_RESULTS:
    import scrapbook as sb
    sb.glue("materialized_feature_values", materialized_feature_values)

### Cleanup

In [ ]:
# Cleaning up the output files. CAUTION: this maybe dangerous if you "reused" the project name.
import shutil
shutil.rmtree(WORKING_DIR, ignore_errors=False)